# Bonus round - videos and GUIs

## Initialization

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

from matplotlib.colors import NoNorm

In [3]:
# convenience function that estimates the difference between 2 images
def diffscore(img1, img2):
    return np.sum(abs(img1-img2))

# convenience function to draw 1 image - no typing 4 lines anymore
def display(img):
    plt.figure()
    plt.axis("off")
    if len(img.shape) == 2:
        plt.imshow(img,cmap='gray',norm=NoNorm())
    elif img.shape[2] == 1:
        plt.imshow(img,cmap='gray',norm=NoNorm())
    else:
        plt.imshow(img)
    plt.show()

## Video processing

All the things you can do with a single image, you can also do with a video, frame-by-frame:

In [7]:
cap = cv2.VideoCapture(0)

In [4]:
ident = lambda x:x

def cap_callback(cap, action):
    ret = True
    while ret:
        ret, img = cap.read()
        img = action(img)
        cv2.imshow("Processed Video", img)
        if cv2.waitKey(1) & 0xff == ord('q'):
            ret = False
    cv2.destroyAllWindows()

In [10]:
cap_callback(cap, ident)

In [9]:
blur = lambda x:cv2.blur(x, (5,5))
cap_callback(cap, blur)

In [8]:
import random
def pepper_salt(img, low=16, high=242):
    out = img.copy()
    random_mat = np.array([[random.randint(0, 255) for j in range(img.shape[1])] for i in range(img.shape[0])])
    pepper_mat = random_mat<low
    pepper_mat = np.stack((pepper_mat,pepper_mat,pepper_mat), axis=2)
    salt_mat = random_mat>high
    salt_mat = np.stack((salt_mat,salt_mat,salt_mat), axis=2)
    
    out = np.where(pepper_mat, 0, out)
    out = np.where(salt_mat, 255, out)
    return out

cap_callback(cap, pepper_salt)

In [11]:
def censor_orange(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (14,100,120), (20,255,255))
    mask = cv2.dilate(mask, np.full((13,13),1))
    out = np.stack([np.where(mask, 0, img[:,:,0]),
                   np.where(mask, 0, img[:,:,1]),
                   np.where(mask, 0, img[:,:,2])], axis=2)
    return out

cap_callback(cap, censor_orange)

## PySimpleGUI

For simple graphical apps in Python, this library is the optimal choice in terms of simplicity.

!pip install PySimpleGUI

In [1]:
import PySimpleGUI as sg

We get to define a window as an array:

In [13]:
layout = [
    [sg.Text("This is a text next to a quit button"), sg.Button("quit")] # 2D array of elements represents layout
]

In [14]:
window = sg.Window("example", layout)

In [15]:
run = True
while run:
    if window.was_closed(): # check if user clicked on the close button
        break
    event, values = window.read() # acquire event and values
    if event == "quit": # if event equals to "quit", do that
        run = False
    window = window.refresh() # refresh window after any updates
window.close() # kill window

If you try to re-run the last snippet itself, nothing will happen. Mind this during the project: the window object is deinitialized after .close(), so you need to create a new object there. **And you need to create a new array for the layout (due to internal limitations)**.

In [16]:
layout = [
    [sg.Text("This is a text next to a quit button"), sg.Button("quit")],
    [sg.Button("I am a button"), sg.Button("I am a button with a key",k="CLICKED")]
]


window = sg.Window("example", layout)

run = True
while run:
    if window.was_closed():
        break
    event, values = window.read()
    print(event, values) # observe that the event is the key of the interacted element
                         # and values are a dictionary of element values if any
    if event == "quit":
        run = False
    window = window.refresh()
window.close()

CLICKED {}
I am a button {}
quit {}


You can interact with objects based on their keys, which can be anything you define:

In [17]:
counter = 0
layout = [
    [sg.Text("This is a text next to a quit button"), sg.Button("quit")],
    [sg.Text(f"This says {counter}",k="THIS_SAYS"), sg.Button("Add one!",k='SOME_NUMBER')]
]

window = sg.Window("example", layout)

run = True
while run:
    if window.was_closed():
        break
    event, values = window.read()
    print(event, values)
    if event == "quit":
        run = False
    elif event == "SOME_NUMBER":
        counter += 1
        window['THIS_SAYS'].update(value = f"This says {counter}") # internally, the window is accessible as a dictionary
                                                                   # and every element exposes the .update function
    window = window.refresh()
window.close()

SOME_NUMBER {}
SOME_NUMBER {}
SOME_NUMBER {}
SOME_NUMBER {}
SOME_NUMBER {}
quit {}


In [21]:
layout = [
    [sg.Text("This is a text next to a quit button"), sg.Button("quit")],
    [sg.Text(f"This says 0",k="THIS_SAYS"), sg.Slider(range=(0,255), k="SOME_NUMBER", default_value=0,size=(20,20),enable_events=True, orientation="horizontal")]
] # Note that here I had to enable slider events for the slider to emit an update every time the value changes.

window = sg.Window("example", layout)

run = True
while run:
    if window.was_closed():
        break
    event, values = window.read()
    print(event, values) # and here you see that the values dictionary is arranged by element keys
    if event == "quit":
        run = False
    elif event == "SOME_NUMBER":
        window['THIS_SAYS'].update(value = f"This says {values['SOME_NUMBER']}")
    window = window.refresh()
window.close()

SOME_NUMBER {'SOME_NUMBER': 2.0}
SOME_NUMBER {'SOME_NUMBER': 0.0}
SOME_NUMBER {'SOME_NUMBER': 3.0}
SOME_NUMBER {'SOME_NUMBER': 5.0}
SOME_NUMBER {'SOME_NUMBER': 7.0}
SOME_NUMBER {'SOME_NUMBER': 10.0}
SOME_NUMBER {'SOME_NUMBER': 2.0}
SOME_NUMBER {'SOME_NUMBER': 20.0}
SOME_NUMBER {'SOME_NUMBER': 17.0}
SOME_NUMBER {'SOME_NUMBER': 22.0}
SOME_NUMBER {'SOME_NUMBER': 29.0}
SOME_NUMBER {'SOME_NUMBER': 32.0}
SOME_NUMBER {'SOME_NUMBER': 43.0}
SOME_NUMBER {'SOME_NUMBER': 51.0}
SOME_NUMBER {'SOME_NUMBER': 61.0}
SOME_NUMBER {'SOME_NUMBER': 70.0}
SOME_NUMBER {'SOME_NUMBER': 75.0}
SOME_NUMBER {'SOME_NUMBER': 82.0}
SOME_NUMBER {'SOME_NUMBER': 90.0}
SOME_NUMBER {'SOME_NUMBER': 95.0}
SOME_NUMBER {'SOME_NUMBER': 99.0}
SOME_NUMBER {'SOME_NUMBER': 102.0}
SOME_NUMBER {'SOME_NUMBER': 99.0}
SOME_NUMBER {'SOME_NUMBER': 119.0}
SOME_NUMBER {'SOME_NUMBER': 126.0}
SOME_NUMBER {'SOME_NUMBER': 136.0}
SOME_NUMBER {'SOME_NUMBER': 146.0}
SOME_NUMBER {'SOME_NUMBER': 158.0}
SOME_NUMBER {'SOME_NUMBER': 170.0}
SOME_NUMBER {

While I really hope that this library finds its way into your personal projects, in the scope of this course we still need one more thing - images!

Of course, we can handle images (although extra conversion is involved):

In [22]:
cap = cv2.VideoCapture(0)
_, img = cap.read()
imgbytes = cv2.imencode('.png', img)[1].tobytes() # this part is necessary for images inside the GUI

layout = [
    [sg.Text("This is a text next to a quit button"), sg.Button("quit")],
    [sg.Image(k="IMAGE", data=imgbytes), sg.Button("Update snap", k="UPD")]
]

window = sg.Window("example", layout)

run = True
while run:
    if window.was_closed():
        break
    event, values = window.read()
    if event == "quit":
        run = False
    elif event == "UPD": # if UPD, update image
        _, img = cap.read()
        imgbytes = cv2.imencode('.png', img)[1].tobytes()
        window['IMAGE'].update(data = imgbytes)
    window = window.refresh()
window.close()

In [23]:
cap = cv2.VideoCapture(0)
_, img = cap.read()
imgbytes = cv2.imencode('.png', img)[1].tobytes()

layout = [
    [sg.Text("This is a text next to a quit button"), sg.Button("quit")],
    [sg.Image(k="IMAGE", data=imgbytes)]
]

window = sg.Window("example", layout)

run = True
while run:
    if window.was_closed():
        break
    event, values = window.read(timeout=50) # timeout = milliseconds to wait until skipping this read action
    if event == "quit":
        run = False
    _, img = cap.read()
    imgbytes = cv2.imencode('.png', img)[1].tobytes()
    window['IMAGE'].update(data = imgbytes)
    window = window.refresh()
window.close()

You can even sort of combine these to play/pause:

In [24]:
cap = cv2.VideoCapture(0)
_, img = cap.read()
imgbytes = cv2.imencode('.png', img)[1].tobytes()

layout = [
    [sg.Text("This is a text next to a quit button"), sg.Button("quit")],
    [sg.Image(k="IMAGE", data=imgbytes), sg.Button("PLAY/PAUSE", k="UPD")]
]

window = sg.Window("example", layout)

run = True
video_playing = True
while run:
    if window.was_closed():
        break
    event, values = window.read(timeout=50) # timeout = milliseconds to wait until skipping this read action
    if event == "quit":
        run = False
    elif event == "UPD":
        video_playing = not video_playing
    if video_playing:
        _, img = cap.read()
        imgbytes = cv2.imencode('.png', img)[1].tobytes()
        window['IMAGE'].update(data = imgbytes)
    window = window.refresh()
window.close()

Now you know how to display images and read values from the window in PySimpleGUI! And you know what to do with those images by this point. Good luck!

P.S PySimpleGUI docs can be found at https://www.pysimplegui.org/en/latest/

Use the searchbar to look up attributes and methods of window elements, most importantly the Button, Slider and Image.